# Fantasy baseball - data mining
The purpose of this database is to see what characteristics are associated with high fantasy ratings, by my hitters' metric that I did for Will.

# 1 Needed libraries

In [82]:
import pandas as pd
import numpy as np
from datetime import datetime

# 2 Read in data

In [83]:
hitters = pd.read_csv("hitterdata2.csv")

In [84]:
hitters.head()

,birthYear,birthMonth,birthDay,nameFirst,nameLast,weight,height,bats,throws,debut,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
0,1954,9,8,Don,Aase,190,75,R,R,1977-07-26,...,0,0,0,0,0,0,0,0,0,0
1,1962,7,4,Johnny,Abrego,185,72,R,R,1985-09-04,...,0,0,0,0,0,0,0,0,0,0
2,1958,9,24,Jim,Acker,210,74,R,R,1983-04-07,...,0,0,0,0,0,0,0,0,0,0
3,1959,1,21,Ricky,Adams,180,74,R,R,1982-09-15,...,5,17,25,0,0,0,0,0,5,8
4,1958,2,23,Juan,Agosto,190,74,L,L,1981-09-07,...,0,0,0,0,0,0,0,0,0,0


In [28]:
hitters.columns

Index(['birthYear', 'birthMonth', 'birthDay', 'nameFirst', 'nameLast',
       'weight', 'height', 'bats', 'throws', 'debut', 'yearID', 'teamID', 'G',
       'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB',
       'HBP', 'SH', 'SF', 'GIDP'],
      dtype='object')

In [29]:
hitters.describe()

,birthYear,birthMonth,birthDay,weight,height,yearID,G,AB,R,H,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
count,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,...,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000,37402.000000
mean,1972.796615,6.674536,15.904417,199.627854,73.510106,2001.466793,55.228036,134.240923,17.916983,35.102107,...,16.977140,2.559088,1.102160,12.910486,25.649564,1.055933,1.195364,1.320732,1.113336,3.013181
std,9.972674,3.441114,8.733498,22.437980,2.281520,9.107584,47.459304,189.118473,28.186831,52.908314,...,27.685799,6.870060,2.461656,21.431183,35.819424,2.833277,2.556699,2.553680,2.044599,4.909781
min,1939.000000,1.000000,1.000000,140.000000,66.000000,1985.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1965.000000,4.000000,8.000000,185.000000,72.000000,1994.000000,16.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1973.000000,7.000000,16.000000,200.000000,74.000000,2002.000000,37.000000,26.000000,2.000000,4.000000,...,1.000000,0.000000,0.000000,1.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1981.000000,10.000000,23.000000,215.000000,75.000000,2009.000000,85.000000,224.000000,27.000000,55.000000,...,24.000000,1.000000,1.000000,19.000000,40.000000,1.000000,1.000000,1.000000,1.000000,4.000000
max,1996.000000,12.000000,31.000000,320.000000,83.000000,2016.000000,163.000000,716.000000,152.000000,262.000000,...,165.000000,110.000000,29.000000,232.000000,223.000000,120.000000,35.000000,39.000000,17.000000,35.000000


In [30]:
hitters.dtypes

birthYear      int64
birthMonth     int64
birthDay       int64
nameFirst     object
nameLast      object
weight         int64
height         int64
bats          object
throws        object
debut         object
yearID         int64
teamID        object
G              int64
AB             int64
R              int64
H              int64
2B             int64
3B             int64
HR             int64
RBI            int64
SB             int64
CS             int64
BB             int64
SO             int64
IBB            int64
HBP            int64
SH             int64
SF             int64
GIDP           int64
dtype: object

The key on this dataset is the player ID and year.

# 3 Cleanup

The following tasks need doing:

* Figure out the primary positions of each player, and exclude those who are primarily pitchers
* Replace the three birth date columns with a single date
* Change the debut column into a date
* Add a column representing the season start date (assumed to be April 10)

In [81]:
# Get primary position
gamescols = ['G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_of', 'G_dh']
prim_pos_index = np.argmax(hitters[gamescols].as_matrix(), axis=1)
prim_pos = []
for i in range(len(hitters)):
    prim_pos.append(gamescols[prim_pos_index[i]][2:])
hitters['prim_pos'] = prim_pos

KeyError: "['G_p' 'G_c' 'G_1b' 'G_2b' 'G_3b' 'G_ss' 'G_lf' 'G_cf' 'G_rf' 'G_of' 'G_dh'] not in index"

In [31]:
# Replace three birth date columns with a single date
hitters['birthDate'] = hitters[['birthYear','birthMonth','birthDay']].apply(lambda s: datetime(*s),axis=1)

In [32]:
hitters = hitters.drop(labels=['birthYear','birthMonth','birthDay'], axis=1)

In [39]:
def makeSeasonDate(x):
    s = '{0}-{1}-{2}'.format(x, 4, 10)
    dt = datetime.strptime(s, '%Y-%m-%d')
    return dt

In [45]:
hitters['seasonDate'] = hitters['yearID'].apply(makeSeasonDate)

In [64]:
hitters['debut'] = hitters['debut'].apply(lambda x: datetime.strptime(x, '%Y-%M-%d'))

In [65]:
hitters.head()

,nameFirst,nameLast,weight,height,bats,throws,debut,yearID,teamID,G,...,BB,SO,IBB,HBP,SH,SF,GIDP,birthDate,seasonDate,Age
0,David,Aardsma,215,75,R,R,2004-01-06 00:04:00,2004,SFN,11,...,0,0,0,0,0,0,0,1981-12-27,2004-04-10,22.0
1,David,Aardsma,215,75,R,R,2004-01-06 00:04:00,2006,CHN,45,...,0,0,0,0,1,0,0,1981-12-27,2006-04-10,24.0
2,David,Aardsma,215,75,R,R,2004-01-06 00:04:00,2007,CHA,25,...,0,0,0,0,0,0,0,1981-12-27,2007-04-10,25.0
3,David,Aardsma,215,75,R,R,2004-01-06 00:04:00,2008,BOS,47,...,0,1,0,0,0,0,0,1981-12-27,2008-04-10,26.0
4,David,Aardsma,215,75,R,R,2004-01-06 00:04:00,2009,SEA,73,...,0,0,0,0,0,0,0,1981-12-27,2009-04-10,27.0


# 4 Feature engineering

The following tasks need doing:

* Calculate player age
* Calculate years in the league as (April 10 of season year - debut date). Not quite right but should be close enough.
* Calculate number of singles
* Calculate batting average
* Calculate on-base percentage
* Calculate slugging percentage
* Calculate OPS
* Calculate scaled statistics - BA, R, HR, RBI, SB
* Calculate JDFS Rating

In [57]:
# Player age
hitters['Age'] = np.floor((hitters['seasonDate'] - hitters['birthDate']) / np.timedelta64(1,'Y'))

In [66]:
# Years of service
hitters['YrsServ'] = np.floor((hitters['seasonDate'] - hitters['debut']) / np.timedelta64(1,'Y'))

In [67]:
# Number of singles
hitters['1B'] = hitters['H'] - hitters['2B'] - hitters['3B'] - hitters['HR']

In [73]:
# Batting average
hitters['BA'] = hitters['H'] / hitters['AB']
hitters['BA'] = hitters['BA'].fillna(0)

In [74]:
# On base percentage
hitters['OBP'] = (hitters['H']+hitters['BB']+hitters['HBP']) / (hitters['AB']+hitters['BB']+hitters['HBP']+hitters['SF'])
hitters['OBP'] = hitters['OBP'].fillna(0)

In [75]:
# Slugging percentage
hitters['SLG'] = (hitters['1B'] + 2 * hitters['2B'] + 3 * hitters['3B'] + 4*hitters['HR']) / hitters['AB']

In [76]:
# On base plus slugging
hitters['OPS'] = hitters['OBP'] + hitters['SLG']

In [77]:
# Scaled statistics
hitters['BA_Scaled'] = hitters.groupby("yearID")['BA'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
hitters['HR_Scaled'] = hitters.groupby("yearID")['HR'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
hitters['R_Scaled'] = hitters.groupby("yearID")['R'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
hitters['RBI_Scaled'] = hitters.groupby("yearID")['RBI'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
hitters['SB_Scaled'] = hitters.groupby("yearID")['SB'].apply(lambda x: (x-min(x))/(max(x)-min(x)))

In [78]:
# JDFS rating
def applyPenalty(x):
    if (x < 50):
        return 0.1
    elif (x >= 300):
        return 1
    else:
        return (0.1 + 1.08* ((x-50)/300))

In [79]:
hitters['Penalty'] = hitters['AB'].apply(applyPenalty)

In [80]:
hitters['AdjStars'] = hitters['Penalty']*(hitters['HR_Scaled'] + hitters['R_Scaled'] + hitters['RBI_Scaled'] + hitters['SB_Scaled'] + hitters['BA_Scaled'])